In [20]:
import pickle

import pandas as pd

from transformers import AutoTokenizer

from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import StratifiedKFold

import torch

import pyarrow as pa
import pyarrow.dataset as ds
from datasets import Dataset

from torch.utils.data import Subset

In [2]:
# Abrindo o objeto dataframe
with open(r'df-tokenizado.pickle', 'rb') as pickledfile:
    df = pickle.load(pickledfile)

In [3]:
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased')

In [4]:
df['Tokenizadas 2'] = [tokenizer.encode(t, truncation='only_first', padding='max_length', max_length=512) for t in df['Texto tratado']]

In [5]:
y = LabelEncoder().fit_transform(X = df['Assunto'].values.reshape(-1,1))

c:\Users\Lucas Alponti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
folds = StratifiedKFold(n_splits=10)

In [7]:
splits = folds.split(df['Tokenizadas 2'], y)

In [8]:
list_splits = []

for f, (t_, v_) in enumerate(splits):
    list_splits.append([f, t_, v_])

In [10]:
df_teste = pd.DataFrame.from_dict(
    {'input_ids': df.loc[:, 'Tokenizadas 2'], 'label': y, 'attention_mask':[[0 if elem == 0 else 1 for elem in row] for row in df.loc[:, 'Tokenizadas 2']] }
)

In [47]:
# Salvando o objeto dataframe
with open(r'df-pre-processado-cloud.pickle', 'wb') as picklefile:
    pickle.dump(df_teste, picklefile, protocol=pickle.HIGHEST_PROTOCOL)

In [48]:
# Salvando o objeto contendo os splits
with open(r'lista-splits.pickle', 'wb') as picklefile:
    pickle.dump(list_splits, picklefile, protocol=pickle.HIGHEST_PROTOCOL)